# Preço Material
> Neste notebook baixamos os dados de materiais da api disponibilizada pelo MGI (dados abertos) e criamos uma base.
>
> A documentação da API do MGI pode ser encontrada em https://dadosabertos.compras.gov.br/swagger-ui/index.html#/

In [1]:
import json
import duckdb
import ipywidgets
from fastprogress.fastprogress import master_bar, progress_bar
import pandas as pd
import requests
import os

/Users/fredguth/Code/sus/sqlmesh/.env/lib/python3.11/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


Agora que já baixamos os Materiais e colocamos no BD, vamos baixar as unidades que são usadas pelos materiais que baixamos.

In [3]:
db = duckdb.connect('./db.db') #copia

In [4]:
db.sql("describe br_mgi__siasg__dev.materiais")

┌─────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name     │ column_type │  null   │   key   │ default │  extra  │
│       varchar       │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ codigoGrupo         │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ nomeGrupo           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ codigoClasse        │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ nomeClasse          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ codigoPdm           │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ nomePdm             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ codigoItem          │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ descricaoItem       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ statusItem          │ BOOLEAN     │ YES     │ NULL    │ NULL  

In [20]:
materiais = list(db.sql("select distinct codigoItem from br_mgi__siasg__dev.materiais where statusItem=true and codigoPdm in (select distinct * from read_csv('../seeds/pdms.csv', header=false, columns={'column0': 'BIGINT'})) order by codigoItem").df()['codigoItem'])
(materiais[:5], len(materiais))

([5410, 6467, 7838, 56235, 111333], 34698)

In [21]:
def fetch_and_save_despesas(codes):
    base_url = 'https://dadosabertos.compras.gov.br/modulo-pesquisa-preco/1_consultarMaterial' 
    
    def save_json(code, anopage):
        response = requests.get(f'{base_url}?codigoItemCatalogo={code}&pagina={page}').json()
        if len(response["resultado"])>0:
            if page==1: os.makedirs(f'../seeds/compras/codigoItemCatalogo={code}', exist_ok=True)
            with open(f'../seeds/compras/codigoItemCatalogo={code}/page_{page}.json', 'w') as f:
                json.dump(response["resultado"], f)
        return response
    
    for code in  progress_bar(codes, leave=False):
        response = save_json(code, 1)
        total_pages = response.get("totalPaginas", 0)
        if total_pages > 1:
            for page in range(2, total_pages + 1):
                save_json(code, page)

In [22]:
fetch_and_save(materiais)

In [ ]:
268236, 448982,273009, 

In [ ]:
import requests

url = "https://dadosabertos.compras.gov.br/modulo-pesquisa-preco/1_consultarMaterial?codigoItemCatalogo=407620&pagina=1"

payload = {}
headers = {
  'Accept': '*/*'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)